# GRAIN 1:
Modelos de clasificación de trigo y maiz, se entrenan dos modelos diferentes para identificar si el grano esta saludable o entra dentro de una de las 6 categorias de grano inaceptable.

In [ ]:
import tensorflow as tf
import os
import shutil
import numpy as np
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
import  imblearn 
from imblearn.over_sampling import  ADASYN
#from imblearn.utils import check_classification_targets


import scipy
import matplotlib.pyplot as plt
from tqdm import tqdm
import PIL
import datetime
#from PIL import image
#import tensorflow-gpu
#from bayes_opt import BayesianOptimization


print(tf.__version__)
tf.debugging.set_log_device_placement(True)



Iniciamos configurando tensorflow para usar la GPU, al igual que tensorboard

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

if len(tf.config.list_physical_devices('GPU'))>0:
    physical_devices = tf.config.list_physical_devices('GPU')
    print(physical_devices)
    tf.config.experimental.set_virtual_device_configuration(physical_devices[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])

if tf.test.gpu_device_name():
    print('GPU encontrada:', tf.test.gpu_device_name())
else:
    print("GPU no encontrada, usando CPU.")



In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_virtual_device_configuration(
                gpu,
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])  # Limitar a 4096 MB (4 GB)
    except RuntimeError as e:
        print(e)


In [ ]:
tf.debugging.set_log_device_placement(False)

## Maize
Generamos dos iteradores de carpetas para los conjuntos de datos de validación y de entrenamiento.

In [ ]:
# Directorios de entrenamiento y validación
train_dir = '../grainspace/pre/maize/train'
validation_dir = '../grainspace/pre/maize/validation'

datagen = ImageDataGenerator(
    rescale=1./255,          # Normaliza los valores de los píxeles
    shear_range=0.2,         # Aplica transformaciones de corte
    zoom_range=0.2,          # Aplica zoom aleatorio
    horizontal_flip=True     # Invierte las imágenes horizontalmente
)
train_generator = datagen.flow_from_directory(
    train_dir,  # Directorio con las imágenes de entrenamiento
    target_size=(150, 150),     # Tamaño de las imágenes
    batch_size=32,              # Tamaño del lote
    class_mode='categorical'         # Tipo de clasificación (binary, categorical, etc.)
)
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)






Definimos la arquitectura del modelo, una red neural convolucional de 4 capas convolucionales, y dos capas densas.

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(len(train_generator.class_indices), activation='softmax')

])

In [ ]:
"""model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),  # Capa de convolución adicional
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(256, activation='relu'),  # Capa densa adicional
    keras.layers.Dense(len(train_generator.class_indices), activation='softmax')
])"""

Compilamos el modeloy realizamos un resumen de la arquitectura:

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
model.summary()

Antes de entrenar el modelo, configuramos el early stopping y el tensorboard, donde el primero permite al modelo detenerse en caso de encontrar que este no mejora con mas epocas, y el segundo muestra el historial de la red en tiempo real.

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=30,
    callbacks=[tensorboard_callback, early_stopping],
    batch_size=32
)

Generamos una evaluación del modelo en conjunto de validación:
donde loss es una metrica que indica la perdida de la red, y accuracy es la proporcion de imagenes clasificadas correctamente

In [ ]:
# Evaluación en conjunto de validación
loss, accuracy = model.evaluate(validation_generator)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

In [ ]:
model.save('maize_classifier_model.h5')


In [ ]:
stop xd

# Wheat

In [ ]:
# Directorios de entrenamiento y validación
train_dir = '../grainspace/pre/wheat/train'
validation_dir = '../grainspace/pre/wheat/validation'

datagen = ImageDataGenerator(
    rescale=1./255,          # Normaliza los valores de los píxeles
    shear_range=0.2,         # Aplica transformaciones de corte
    zoom_range=0.2,          # Aplica zoom aleatorio
    horizontal_flip=True     # Invierte las imágenes horizontalmente
)
train_generator = datagen.flow_from_directory(
    train_dir,  # Directorio con las imágenes de entrenamiento
    target_size=(150, 150),     # Tamaño de las imágenes
    batch_size=32,              # Tamaño del lote
    class_mode='categorical'         # Tipo de clasificación (binary, categorical, etc.)
)
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)



In [ ]:
"""model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(len(train_generator.class_indices), activation='softmax')

])"""

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),  # Capa de convolución adicional
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(256, activation='relu'),  # Capa densa adicional
    keras.layers.Dense(len(train_generator.class_indices), activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=30,
    callbacks=[early_stopping, tensorboard_callback],
    batch_size=32
)

In [ ]:
# Evaluación en conjunto de validación
loss, accuracy = model.evaluate(validation_generator)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

In [ ]:
model.save('wheat_classifier_model.h5')

